In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import numpy as np
import os
import cv2
import GPUtil
import matplotlib.pyplot as plt                        
%matplotlib inline  

# In a multi GPU server, chose which to use:
NUMBER_OF_GPUS_TO_USE = 1
Availability=GPUtil.getAvailability(GPUtil.getGPUs())
all_gpus = np.arange(3)
available_gpu_indexes = [x for x in all_gpus if Availability[x]]
# Set CUDA_DEVICE_ORDER so the IDs assigned by CUDA match those from nvidia-smi
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# Set CUDA_VISIBLE_DEVICES to mask out all other GPUs than the first NUMBER_OF_GPUS_TO_USE available device id
os.environ["CUDA_VISIBLE_DEVICES"] = ','.join(np.array(available_gpu_indexes[:NUMBER_OF_GPUS_TO_USE]).astype(str))

import tensorflow as tf
config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))

import keras
from keras.backend.tensorflow_backend import set_session

set_session(tf.Session(config=config))

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Check if its a human face

In [3]:
# extract pre-trained face detector
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt.xml')
# returns "True" if face is detected in image stored at img_path
def face_detector(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    return len(faces) > 0

### Confirm the image is actually a dog

The ability of this process is limited by the dataset

In [4]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image                  
from tqdm import tqdm

# define ResNet50 model
ResNet50_model = ResNet50(weights='imagenet')

In [19]:
def path_to_tensor(img_path, expand=True, **kwargs):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, **kwargs)
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    if expand:
        return np.expand_dims(x, axis=0)
    else:
        return x

def paths_to_tensor(img_paths, **kwargs):
    list_of_tensors = [path_to_tensor(img_path, **kwargs) for img_path in img_paths]
    return np.vstack(list_of_tensors)

In [6]:
from keras.applications.resnet50 import preprocess_input, decode_predictions

def ResNet50_predict_labels(img_path):
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path))
    return np.argmax(ResNet50_model.predict(img))

### returns "True" if a dog is detected in the image stored at img_path
def dog_detector(img_path):
    prediction = ResNet50_predict_labels(img_path)
    return ((prediction <= 268) & (prediction >= 151)) 

### Use transfer learning to reduce training time

Model architecture

In [95]:
### Define your architecture.
from keras.models import Sequential
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.core import Dense

Resnet50_model = Sequential()
Resnet50_model.add(GlobalAveragePooling2D(input_shape=(7, 7, 2048)))
Resnet50_model.add(Dense(120, activation='softmax'))

In [96]:
Resnet50_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_5 ( (None, 2048)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 120)               245880    
Total params: 245,880
Trainable params: 245,880
Non-trainable params: 0
_________________________________________________________________


#### Compile and test

##### Load data

In [7]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from glob import glob

data = load_files("/notebooks/Practice/imgnet-dogbreed/Images/")

In [8]:
data.keys()

dict_keys(['DESCR', 'target_names', 'target', 'data', 'filenames'])

In [9]:
train_files, test_files, train_targets, test_targets = train_test_split(data['filenames'], data['target'], test_size=0.2)
test_files, validation_files, test_targets, validation_targets = train_test_split(test_files, test_targets, test_size=0.5)
train_targets = np_utils.to_categorical(np.array(train_targets), 120)
test_targets = np_utils.to_categorical(np.array(test_targets), 120)
validation_targets = np_utils.to_categorical(np.array(validation_targets), 120)

dog_names = [item.split('/')[-1] for item in sorted(glob("/notebooks/Practice/imgnet-dogbreed/Images/*"))]

In [10]:
test_files[0]

'/notebooks/Practice/imgnet-dogbreed/Images/n02097474-Tibetan_terrier/n02097474_4929.jpg'

In [11]:
np.argmax(test_targets)

49

In [12]:
data['target_names'][32]

'n02093859-Kerry_blue_terrier'

In [13]:
# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, validation_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d test dog images.'% len(validation_files))
print('There are %d test dog images.'% len(test_files))

There are 120 total dog categories.
There are 20580 total dog images.

There are 16464 training dog images.
There are 2058 test dog images.
There are 2058 test dog images.


Create bottleneck features

```python
### Obtain bottleneck features from another pre-trained CNN.
bottleneck_features = np.load('AIND2-dog-project/bottleneck_features/DogResnet50Data.npz')
train_DogResnet50 = bottleneck_features['train']
valid_DogResnet50 = bottleneck_features['valid']
test_DogResnet50 = bottleneck_features['test']

set(zip(bottleneck_features.keys(), [np.shape(bottleneck_features[key]) for key in bottleneck_features.keys()]))

{('filenames_test', (836,)),
 ('filenames_train', (6680,)),
 ('filenames_valid', (835,)),
 ('test', (836, 1, 1, 2048)),
 ('test_OH', (836, 133)),
 ('train', (6680, 1, 1, 2048)),
 ('train_OH', (6680, 133)),
 ('valid', (835, 1, 1, 2048)),
 ('valid_OH', (835, 133))}
```

In [25]:
from keras.engine.input_layer import Input
from keras.models import Model

In [26]:
input_img = Input(shape=(360, 360, 3))

model_Inception = keras.applications.InceptionV3(
    include_top=True,weights='imagenet',
    input_shape=(360, 360, 3),
    input_tensor=input_img)
extracted_features = model_Inception.layers[-1].input
extraction_model = Model(inputs=input_img,outputs=extracted_features)

In [27]:
%time train_features = extraction_model.predict(preprocess_input(paths_to_tensor(train_files, target_size=(360, 360))))

CPU times: user 3min 34s, sys: 55.6 s, total: 4min 29s
Wall time: 4min 19s


In [28]:
%time test_features = extraction_model.predict(preprocess_input(paths_to_tensor(test_files, target_size=(360, 360))))

CPU times: user 26.8 s, sys: 5.51 s, total: 32.3 s
Wall time: 31.1 s


In [29]:
%time validation_features = extraction_model.predict(preprocess_input(paths_to_tensor(validation_files, target_size=(360, 360))))

CPU times: user 26.3 s, sys: 5.11 s, total: 31.4 s
Wall time: 30.1 s


In [30]:
[[np.shape(train_features), np.shape(test_features), np.shape(validation_features)],
 [np.shape(train_targets), np.shape(test_targets), np.shape(validation_targets)]]

[[(16464, 2048), (2058, 2048), (2058, 2048)],
 [(16464, 120), (2058, 120), (2058, 120)]]

In [31]:
mkdir InceptionV3

In [78]:
train_features = np.load('InceptionV3/train_features.npy')
test_features = np.load('InceptionV3/test_features.npy')
valid_features = np.load('InceptionV3/validation_features.npy')

train_targets = np.load('InceptionV3/train_targets.npy')
test_targets = np.load('InceptionV3/test_targets.npy')
validation_targets = np.load('InceptionV3/validation_targets.npy')

train_files = np.load('InceptionV3/train_files.npy')
test_files = np.load('InceptionV3/test_files.npy')
validation_files = np.load('InceptionV3/validation_files.npy')

In [33]:
np.save('InceptionV3/train_features', train_features)
np.save('InceptionV3/test_features', test_features)
np.save('InceptionV3/validation_features', validation_features)
np.save('InceptionV3/train_targets', train_targets)
np.save('InceptionV3/test_targets', test_targets)
np.save('InceptionV3/validation_targets', validation_targets)

np.save('InceptionV3/train_files',train_files)
np.save('InceptionV3/test_files',test_files)
np.save('InceptionV3/validation_files',validation_files)

In [36]:
from keras.callbacks import ModelCheckpoint
from keras.layers.core import Dense

In [37]:
input_features = Input(shape=(2048,))
out = Dense(units=120,activation='softmax',use_bias=True)(input_features)
Iv3_model = Model(inputs=input_features,outputs=out)

In [40]:
Iv3_model.compile(optimizer=keras.optimizers.Adam(lr=0.01),
              loss=keras.losses.categorical_crossentropy,
              metrics=[keras.metrics.categorical_accuracy])
checkpointer_iv3 = ModelCheckpoint(filepath='InceptionV3/weights.best.InceptionV3.hdf5', 
                               verbose=0, save_best_only=True)
Iv3_model.fit(train_features, train_targets, 
      validation_data=(validation_features, validation_targets),
      epochs=200, batch_size=200, callbacks=[checkpointer_iv3], verbose=0)

# np.expand_dims(np.expand_dims(validation_targets,1),1))

In [44]:
### Load the model weights with the best validation loss.
Iv3_model.load_weights('InceptionV3/weights.best.InceptionV3.hdf5')

In [41]:
### Calculate classification accuracy on the test dataset.
predictions = np.argmax(Iv3_model.predict(test_features), axis=1)
actual = np.argmax(test_targets, axis=1)
# Report test accuracy
test_accuracy = 100*np.sum(predictions==actual)/len(predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 0.9232%
